In [1]:
import h5py
import numpy as np
import pandas as pd
import os
from util import transMap,getDir,transDir,sqvz_zip,showMap,ModeTransfer
Path = "/mnt/d/data/pilot7-04-Aug-2022-1"
fileNames = os.listdir(Path)

In [2]:
GT = pd.read_csv("Table.csv")

In [3]:
g1_gt = GT.ghost1.values
g2_gt = GT.ghost2.values

In [4]:
df = pd.read_csv("pilot7-04-Aug-2022-1.csv")

In [17]:
np.all(df.loc[df['DayTrial'].str.startswith("1-1"),"ifscared1"].values == g1_gt)

True

In [18]:
np.all(df.loc[df['DayTrial'].str.startswith("1-1"),"ifscared2"].values == g2_gt)

True

In [4]:
file = h5py.File(f"{Path}/{fileNames[0]}", 'r')
data = file['data']
g1 = data['ghosts']['mode'][:,0]
g2 = data['ghosts']['mode'][:,1]
g3 = data['ghosts']['mode'][:,2]
g4 = data['ghosts']['mode'][:,3]
g1_new = np.ones(len(g1)) * -1
g2_new = np.ones(len(g2)) * -1
g3_new = np.ones(len(g3)) * -1
g4_new = np.ones(len(g4)) * -1

In [5]:
# flash parameter
i_ = np.floor((data['energizer']['duration'][:]-data['energizer']['count'][:])/ \
    data['energizer']['flashInterval'][:])
# chasing or corner parameter
dx = np.squeeze(data['pacMan']['tile_x'][:]) - (data['ghosts']['tile_x'][:,1] + data['ghosts']['dir_x'][:,1])
dy = np.squeeze(data['pacMan']['tile_y'][:]) - (data['ghosts']['tile_y'][:,1] + data['ghosts']['dir_y'][:,1])
dist = dx*dx+dy*dy

In [6]:
# index = data['ghosts']['scared'][:,0] == 0, \
#         np.array([g1_i == 0 or g1_i == 4 or g1_i == 5 for g1_i in g1]))
# g1_new[index] = 1 # chasing pacman
# g1_new[np.array([g1_i == 1 or g1_i == 2 or g1_i == 3 for g1_i in g1])] = 3; # dead ghosts (include ghosts are being eaten)
# index = np.logical_and(np.squeeze(i_ > 2*data['energizer']['flashes'][:]-1), \
#         data['ghosts']['scared'][:,0] == 1)
# g1_new[index] = 4; # scared ghosts
# index = np.logical_and(np.squeeze(i_ <= 2*data['energizer']['flashes'][:]-1), \
#         data['ghosts']['scared'][:,0] == 1)
# g1_new[index] = 5; # flash scared ghosts

In [7]:
# index = np.array([g2_i == 0 or g2_i == 4 or g2_i == 5 for g2_i in g2])
# g2_new[(dist >= 64) & (data['ghosts']['scared'][:,1] == 0) & index] = 1; # chasing pacman
# g2_new[(dist < 64) & (data['ghosts']['scared'][:,1] == 0) & index] = 2; # going corner
# index = np.array([g2_i == 1 or g2_i == 2 or g2_i == 3 for g2_i in g2])
# g2_new[index] = 3; # dead ghosts (include ghosts are being eaten)
# g2_new[np.squeeze(i_ > 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,1] == 1)] = 4; # scared ghosts
# g2_new[np.squeeze(i_ <= 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,1] == 1)] = 5; # flash scared ghosts


In [8]:
index = (g1 == 0) | (g1 == 4) | (g1 == 5)
g1_new[(data['ghosts']['scared'][:,0] == 0) & index] = 1 # chasing pacman
g1_new[(g1 == 1) | (g1 == 2) | (g1 == 3)] = 3; # dead ghosts (include ghosts are being eaten)
g1_new[np.squeeze(i_ > 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,0] == 1)] = 4; # scared ghosts
g1_new[np.squeeze(i_ <= 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,0] == 1)] = 5; # flash scared ghosts

index = np.array([g2_i == 0 or g2_i == 4 or g2_i == 5 for g2_i in g2])
g2_new[(dist >= 64) & (data['ghosts']['scared'][:,1] == 0) & index] = 1; # chasing pacman
g2_new[(dist < 64) & (data['ghosts']['scared'][:,1] == 0) & index] = 2; # going corner
index = np.array([g2_i == 1 or g2_i == 2 or g2_i == 3 for g2_i in g2])
g2_new[index] = 3; # dead ghosts (include ghosts are being eaten)
g2_new[np.squeeze(i_ > 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,1] == 1)] = 4; # scared ghosts
g2_new[np.squeeze(i_ <= 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,1] == 1)] = 5; # flash scared ghosts

In [9]:
print(np.all(g1_new == g1_gt))
print(np.all(g2_new == g2_gt))

True
True


In [25]:
a = np.array([0,1,1,1,0,2,3,3,0])
index = a > 0

In [26]:
index

array([False,  True,  True,  True, False,  True,  True,  True, False])

In [27]:
a == 1

array([False,  True,  True,  True, False, False, False, False, False])

In [22]:
np.logical_and(a == 1,index)

array([False,  True,  True,  True, False, False, False, False, False])

In [31]:
a == 1 & index

array([ True,  True,  True,  True,  True, False, False, False,  True])

In [27]:
with h5py.File(f"{Path}/{fileNames[0]}", 'r') as file:
    data = file['data']
    g1 = data['ghosts']['mode'][:,0]
    g2 = data['ghosts']['mode'][:,1]
    g3 = data['ghosts']['mode'][:,2]
    g4 = data['ghosts']['mode'][:,3]
    g1_new = g1
    g2_new = g2
    g3_new = g3
    g4_new = g4
    # flash parameter
    i_ = np.floor((data['energizer']['duration'][:]-data['energizer']['count'][:])/ \
        data['energizer']['flashInterval'][:])
    # chasing or corner parameter
    dx = np.squeeze(data['pacMan']['tile_x'][:]) - (data['ghosts']['tile_x'][:,2] + data['ghosts']['dir_x'][:,2])
    dy = np.squeeze(data['pacMan']['tile_y'][:]) - (data['ghosts']['tile_y'][:,2] + data['ghosts']['dir_y'][:,2])
    dist = dx*dx+dy*dy

    index = np.array([g1_i == 0 or g1_i == 4 or g1_i == 5 for g1_i in g1])
    g1_new[data['ghosts']['scared'][:,0] == 0 & index] = 1 # chasing pacman
    index = np.array([g1_i == 1 or g1_i == 2 or g1_i == 3 for g1_i in g1])
    g1_new[index] = 3; # dead ghosts (include ghosts are being eaten)
    g1_new[np.squeeze(i_ > 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,0] == 1)] = 4; # scared ghosts
    g1_new[np.squeeze(i_ <= 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,0] == 1)] = 5; # flash scared ghosts

    index = np.array([g2_i == 0 or g2_i == 4 or g2_i == 5 for g2_i in g2])
    g2_new[dist >= 64 & (data['ghosts']['scared'][:,1] == 0) & index] = 1; # chasing pacman
    g2_new[dist < 64 & (data['ghosts']['scared'][:,1] == 0) & index] = 2; # going corner
    index = np.array([g2_i == 1 or g2_i == 2 or g2_i == 3 for g2_i in g2])
    g2_new[index] = 3; # dead ghosts (include ghosts are being eaten)
    g2_new[np.squeeze(i_ > 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,1] == 1)] = 4; # scared ghosts
    g2_new[np.squeeze(i_ <= 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,1] == 1)] = 5; # flash scared ghosts

    index = np.array([g3_i == 0 or g3_i == 4 or g3_i == 5 for g3_i in g3])
    g3_new[data['ghosts']['scared'][:,2] == 0 & index] = 1 # chasing pacman
    index = np.array([g3_i == 1 or g3_i == 2 or g3_i == 3 for g3_i in g3])
    g3_new[index] = 3; # dead ghosts (include ghosts are being eaten)
    g3_new[np.squeeze(i_ > 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,2] == 1)] = 4; # scared ghosts
    g3_new[np.squeeze(i_ <= 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,2] == 1)] = 5; # flash scared ghosts
    
    index = np.array([g4_i == 0 or g4_i == 4 or g4_i == 5 for g4_i in g4])
    g4_new[data['ghosts']['scared'][:,3] == 0 & index] = 1 # chasing pacman
    index = np.array([g4_i == 1 or g4_i == 2 or g4_i == 3 for g4_i in g4])
    g4_new[index] = 3; # dead ghosts (include ghosts are being eaten)
    g4_new[np.squeeze(i_ > 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,3] == 1)] = 4; # scared ghosts
    g4_new[np.squeeze(i_ <= 2*data['energizer']['flashes'][:]-1) & (data['ghosts']['scared'][:,3] == 1)] = 5; # flash scared ghosts



In [28]:
data = dict({'DayTrial': fileNames[0][0:-4],
         'ifscared1': ifsacred1,
         'ifscared2': ifsacred2,
         'ifscared3': ifsacred3,
         'ifscared4': ifsacred4})

In [30]:
pd.DataFrame(data)

,DayTrial,ifscared1,ifscared2,ifscared3,ifscared4
0,1-1-pilot7-04-Aug-2022,3,3,3,3
1,1-1-pilot7-04-Aug-2022,3,3,3,3
2,1-1-pilot7-04-Aug-2022,3,3,3,3
3,1-1-pilot7-04-Aug-2022,3,3,3,3
4,1-1-pilot7-04-Aug-2022,3,3,3,3
...,...,...,...,...,...
3215,1-1-pilot7-04-Aug-2022,3,3,3,3
3216,1-1-pilot7-04-Aug-2022,3,3,3,3
3217,1-1-pilot7-04-Aug-2022,3,3,3,3
3218,1-1-pilot7-04-Aug-2022,3,3,3,3


In [31]:
dataFrame = pd.DataFrame()
for fileName in fileNames:
    if not fileName.endswith(".mat"):
        continue
    with h5py.File(f"{Path}/{fileName}", 'r') as file:
        data = file['data']
        pacMan_x = data['pacMan']['tile_x'][:]
        pacMan_y = data['pacMan']['tile_y'][:]
        ghosts_x = data['ghosts']['tile_x'][:]
        ghosts_y = data['ghosts']['tile_y'][:]
        up = data['direction']['up'][:]
        down = data['direction']['down'][:]
        right = data['direction']['right'][:]
        left = data['direction']['left'][:]
        dirEnum = np.int8(data['pacMan']['dirEnum'][:])
        Map = data['gameMap']['currentTiles'][:]
        ifsacred1,ifsacred2,ifsacred3,ifsacred4 = ModeTransfer(data)
    data = dict({'DayTrial': fileName[0:-4],
             'Step': range(0,len(up)),
             'pacManPos': sqvz_zip(pacMan_x,pacMan_y), 
             'ghost1Pos': sqvz_zip(ghosts_x[:,0],ghosts_y[:,0]),
             'ghost2Pos': sqvz_zip(ghosts_x[:,1],ghosts_y[:,1]),
             'ghost3Pos': sqvz_zip(ghosts_x[:,2],ghosts_y[:,2]),
             'ghost4Pos': sqvz_zip(ghosts_x[:,3],ghosts_y[:,3]),
             'ifscared1': ifsacred1,
             'ifscared2': ifsacred2,
             'ifscared3': ifsacred3,
             'ifscared4': ifsacred4,
             'pacman_dir': transDir(dirEnum),
             'JoyStick': getDir(up, down, right, left),
             'Map': transMap(Map)})
    df = pd.DataFrame(data)
    dataFrame = pd.concat([dataFrame,df],ignore_index=True)
dataFrame.to_csv("pilot7-04-Aug-2022-1.csv")

In [33]:
dataFrame.head(10)

,DayTrial,Step,pacManPos,ghost1Pos,ghost2Pos,ghost3Pos,ghost4Pos,ifscared1,ifscared2,ifscared3,ifscared4,pacman_dir,JoyStick,Map
0,1-1-pilot7-04-Aug-2022,0,"(15, 9)","(14, 18)","(15, 19)","(14, 17)","(14, 20)",3,3,3,3,,,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."
1,1-1-pilot7-04-Aug-2022,1,"(15, 9)","(14, 18)","(15, 19)","(14, 17)","(14, 20)",3,3,3,3,,,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."
2,1-1-pilot7-04-Aug-2022,2,"(15, 9)","(14, 18)","(15, 19)","(14, 17)","(14, 20)",3,3,3,3,,,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."
3,1-1-pilot7-04-Aug-2022,3,"(15, 9)","(14, 18)","(15, 19)","(14, 17)","(14, 20)",3,3,3,3,,right,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."
4,1-1-pilot7-04-Aug-2022,4,"(15, 9)","(14, 18)","(15, 19)","(14, 17)","(14, 20)",3,3,3,3,right,right,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."
5,1-1-pilot7-04-Aug-2022,5,"(15, 9)","(14, 18)","(15, 19)","(14, 17)","(14, 20)",3,3,3,3,right,right,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."
6,1-1-pilot7-04-Aug-2022,6,"(15, 9)","(14, 18)","(15, 19)","(14, 17)","(14, 20)",3,3,3,3,right,right,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."
7,1-1-pilot7-04-Aug-2022,7,"(15, 9)","(14, 18)","(15, 19)","(14, 17)","(14, 20)",3,3,3,3,right,right,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."
8,1-1-pilot7-04-Aug-2022,8,"(15, 9)","(14, 18)","(15, 19)","(14, 17)","(14, 20)",3,3,3,3,right,right,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."
9,1-1-pilot7-04-Aug-2022,9,"(15, 9)","(14, 18)","(15, 19)","(14, 17)","(14, 20)",3,3,3,3,right,right,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."


In [4]:
map_i = dataFrame.loc[0,"Map"]
map_i = np.reshape(map_i,(36,29))
showMap(map_i)

_____________________________
_____________________________
_____________________________
|||||||||||||||||||||||||||||
|      ......|||          . |
| |||| |||||.|||.||||| |||| |
| |||| |||||.||| |||||.|||| |
| |||| |||||.||| |||||.|||| |
| .... .. ....              |
| |||| || ||||||||| || |||| |
| |||| || ||||||||| ||.|||| |
|      ||    |||    ||      |
|||||| ||||| ||| ||||| ||||||
_____| ||||| ||| ||||| |_____
_____| ||           || |_____
_____| || |||---||| || |_____
|||||| || |_______| || ||||||
          |_______|          
|||||| || |_______| || ||||||
_____| || ||||||||| || |_____
_____|.||           || |_____
_____| ||.||||||||| || |_____
|||||| ||.||||||||| || ||||||
|            |||            |
| |||| ||||| ||| ||||| |||| |
|.|||| ||||| ||| ||||| |||| |
|.||||                 |||| |
| |||| || ||||||||| || |||| |
| |||| || ||||||||| || |||| |
|      ||.   |||    ||      |
|.|||||||||| ||| |||||||||| |
|.||||||||||.||| |||||||||| |
|   . .. .. ..              |
||||||||||

In [ ]:
df1 = pd.read_csv("data/omegaL-03-Dec-2020-1pFlip.csv")

In [ ]:
df1.columns.values

In [ ]:
df1.head(5)